In [1]:
import pandas as pd

In [3]:
df = pd.read_csv(r"C:\Users\assis\Downloads\MPS LSOA Level Crime (Historical) (1).csv")

In [5]:
df_long = df.melt(
    id_vars=['LSOA Code', 'LSOA Name', 'Borough', 'Major Category', 'Minor Category'],
    var_name='Month',
    value_name='Crime_Count'
)

# Convert Month column to datetime
df_long['Month'] = pd.to_datetime(df_long['Month'], format='%Y%m')

# Sort and clean
df_long = df_long.sort_values(['LSOA Code', 'Month'])

In [7]:
df_long

,LSOA Code,LSOA Name,Borough,Major Category,Minor Category,Month,Crime_Count
1581678,E01000006,Barking and Dagenham 016A,E09000002,ARSON AND CRIMINAL DAMAGE,ARSON,2010-04-01,0
1581679,E01000006,Barking and Dagenham 016A,E09000002,ARSON AND CRIMINAL DAMAGE,CRIMINAL DAMAGE,2010-04-01,1
1581680,E01000006,Barking and Dagenham 016A,E09000002,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,2010-04-01,0
1581681,E01000006,Barking and Dagenham 016A,E09000002,BURGLARY,BURGLARY IN A DWELLING,2010-04-01,3
1581682,E01000006,Barking and Dagenham 016A,E09000002,DRUG OFFENCES,POSSESSION OF DRUGS,2010-04-01,1
...,...,...,...,...,...,...,...
17172499,E01035722,Westminster 024G,E09000033,VEHICLE OFFENCES,INTERFERING WITH A MOTOR VEHICLE,2022-11-01,0
17172500,E01035722,Westminster 024G,E09000033,VEHICLE OFFENCES,THEFT FROM A VEHICLE,2022-11-01,1
17172501,E01035722,Westminster 024G,E09000033,VEHICLE OFFENCES,THEFT OR UNAUTH TAKING OF A MOTOR VEH,2022-11-01,0
17172502,E01035722,Westminster 024G,E09000033,VIOLENCE AGAINST THE PERSON,VIOLENCE WITH INJURY,2022-11-01,2


In [9]:
df_grouped = (
    df_long
    .groupby(['LSOA Code', 'LSOA Name', 'Borough', 'Major Category', 'Month'], as_index=False)
    ['Crime_Count']
    .sum()
)

df_grouped = df_grouped.sort_values(['LSOA Code', 'Major Category', 'Month'])

df_grouped

,LSOA Code,LSOA Name,Borough,Major Category,Month,Crime_Count
0,E01000006,Barking and Dagenham 016A,E09000002,ARSON AND CRIMINAL DAMAGE,2010-04-01,1
1,E01000006,Barking and Dagenham 016A,E09000002,ARSON AND CRIMINAL DAMAGE,2010-05-01,3
2,E01000006,Barking and Dagenham 016A,E09000002,ARSON AND CRIMINAL DAMAGE,2010-06-01,0
3,E01000006,Barking and Dagenham 016A,E09000002,ARSON AND CRIMINAL DAMAGE,2010-07-01,2
4,E01000006,Barking and Dagenham 016A,E09000002,ARSON AND CRIMINAL DAMAGE,2010-08-01,0
...,...,...,...,...,...,...
7563059,E01035722,Westminster 024G,E09000033,VIOLENCE AGAINST THE PERSON,2022-07-01,5
7563060,E01035722,Westminster 024G,E09000033,VIOLENCE AGAINST THE PERSON,2022-08-01,4
7563061,E01035722,Westminster 024G,E09000033,VIOLENCE AGAINST THE PERSON,2022-09-01,2
7563062,E01035722,Westminster 024G,E09000033,VIOLENCE AGAINST THE PERSON,2022-10-01,4


In [11]:
df_wide = df_grouped.pivot_table(
    index=['LSOA Code', 'LSOA Name', 'Borough', 'Month'],
    columns='Major Category',
    values='Crime_Count',
    aggfunc='sum',  # just in case there are duplicates
    fill_value=0    # replace missing values with 0 (no crime that month)
)

df_wide.columns.name = None
df_wide = df_wide.reset_index()

df_wide

,LSOA Code,LSOA Name,Borough,Month,ARSON AND CRIMINAL DAMAGE,BURGLARY,DRUG OFFENCES,MISCELLANEOUS CRIMES AGAINST SOCIETY,POSSESSION OF WEAPONS,PUBLIC ORDER OFFENCES,ROBBERY,THEFT,VEHICLE OFFENCES,VIOLENCE AGAINST THE PERSON
0,E01000006,Barking and Dagenham 016A,E09000002,2010-04-01,1,3,1,0,0,0,2,2,2,3
1,E01000006,Barking and Dagenham 016A,E09000002,2010-05-01,3,0,3,0,0,0,0,1,0,2
2,E01000006,Barking and Dagenham 016A,E09000002,2010-06-01,0,1,0,0,0,0,0,0,1,0
3,E01000006,Barking and Dagenham 016A,E09000002,2010-07-01,2,1,0,0,0,0,0,1,3,4
4,E01000006,Barking and Dagenham 016A,E09000002,2010-08-01,0,1,1,0,0,0,0,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758171,E01035722,Westminster 024G,E09000033,2022-07-01,0,0,1,0,0,2,1,3,2,5
758172,E01035722,Westminster 024G,E09000033,2022-08-01,0,3,2,0,0,0,0,5,0,4
758173,E01035722,Westminster 024G,E09000033,2022-09-01,0,1,0,0,0,0,0,6,3,2
758174,E01035722,Westminster 024G,E09000033,2022-10-01,1,4,0,0,0,0,0,6,2,4


In [17]:
df_wide['Borough'].unique()

array(['E09000002', 'E09000003', 'E09000004', 'E09000005', 'E09000006',
       'E09000007', 'E09000008', 'E09000009', 'E09000010', 'E09000011',
       'E09000012', 'E09000013', 'E09000014', 'E09000015', 'E09000016',
       'E09000017', 'E09000018', 'E09000019', 'E09000020', 'E09000021',
       'E09000022', 'E09000023', 'E09000024', 'E09000025', 'E09000026',
       'E09000027', 'E09000028', 'E09000029', 'E09000030', 'E09000031',
       'E09000032', 'E09000033'], dtype=object)

In [19]:
borough_lookup = {
    "E09000002": "Barking and Dagenham",
    "E09000003": "Barnet",
    "E09000004": "Bexley",
    "E09000005": "Brent",
    "E09000006": "Bromley",
    "E09000007": "Camden",
    "E09000008": "Croydon",
    "E09000009": "Ealing",
    "E09000010": "Enfield",
    "E09000011": "Greenwich",
    "E09000012": "Hackney",
    "E09000013": "Hammersmith and Fulham",
    "E09000014": "Haringey",
    "E09000015": "Harrow",
    "E09000016": "Havering",
    "E09000017": "Hillingdon",
    "E09000018": "Hounslow",
    "E09000019": "Islington",
    "E09000020": "Kensington and Chelsea",
    "E09000021": "Kingston upon Thames",
    "E09000022": "Lambeth",
    "E09000023": "Lewisham",
    "E09000024": "Merton",
    "E09000025": "Newham",
    "E09000026": "Redbridge",
    "E09000027": "Richmond upon Thames",
    "E09000028": "Southwark",
    "E09000029": "Sutton",
    "E09000030": "Tower Hamlets",
    "E09000031": "Waltham Forest",
    "E09000032": "Wandsworth",
    "E09000033": "Westminster"
}


In [25]:
len(borough_lookup)

32

In [21]:
df_wide["Borough"] = df_wide["Borough"].map(borough_lookup)
df_wide

,LSOA Code,LSOA Name,Borough,Month,ARSON AND CRIMINAL DAMAGE,BURGLARY,DRUG OFFENCES,MISCELLANEOUS CRIMES AGAINST SOCIETY,POSSESSION OF WEAPONS,PUBLIC ORDER OFFENCES,ROBBERY,THEFT,VEHICLE OFFENCES,VIOLENCE AGAINST THE PERSON
0,E01000006,Barking and Dagenham 016A,Barking and Dagenham,2010-04-01,1,3,1,0,0,0,2,2,2,3
1,E01000006,Barking and Dagenham 016A,Barking and Dagenham,2010-05-01,3,0,3,0,0,0,0,1,0,2
2,E01000006,Barking and Dagenham 016A,Barking and Dagenham,2010-06-01,0,1,0,0,0,0,0,0,1,0
3,E01000006,Barking and Dagenham 016A,Barking and Dagenham,2010-07-01,2,1,0,0,0,0,0,1,3,4
4,E01000006,Barking and Dagenham 016A,Barking and Dagenham,2010-08-01,0,1,1,0,0,0,0,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758171,E01035722,Westminster 024G,Westminster,2022-07-01,0,0,1,0,0,2,1,3,2,5
758172,E01035722,Westminster 024G,Westminster,2022-08-01,0,3,2,0,0,0,0,5,0,4
758173,E01035722,Westminster 024G,Westminster,2022-09-01,0,1,0,0,0,0,0,6,3,2
758174,E01035722,Westminster 024G,Westminster,2022-10-01,1,4,0,0,0,0,0,6,2,4


In [23]:
df_borough = df_wide.groupby(['Borough', 'Month'], as_index=False).sum(numeric_only=True)
df_borough

,Borough,Month,ARSON AND CRIMINAL DAMAGE,BURGLARY,DRUG OFFENCES,MISCELLANEOUS CRIMES AGAINST SOCIETY,POSSESSION OF WEAPONS,PUBLIC ORDER OFFENCES,ROBBERY,THEFT,VEHICLE OFFENCES,VIOLENCE AGAINST THE PERSON
0,Barking and Dagenham,2010-04-01,214,167,85,22,22,29,58,323,252,375
1,Barking and Dagenham,2010-05-01,195,160,71,13,12,37,83,357,234,393
2,Barking and Dagenham,2010-06-01,229,181,87,15,12,30,70,357,262,396
3,Barking and Dagenham,2010-07-01,225,183,82,13,16,31,74,431,227,379
4,Barking and Dagenham,2010-08-01,209,199,107,17,4,35,71,338,242,356
...,...,...,...,...,...,...,...,...,...,...,...,...
4859,Westminster,2022-07-01,171,186,312,37,38,312,321,3946,341,1044
4860,Westminster,2022-08-01,202,265,342,29,45,354,275,3058,326,974
4861,Westminster,2022-09-01,171,274,279,31,45,270,179,2823,276,818
4862,Westminster,2022-10-01,213,257,364,25,37,288,325,4555,373,923


In [35]:
london_trend = df_borough.groupby('Month').mean(numeric_only=True).reset_index()
london_trend

,Month,ARSON AND CRIMINAL DAMAGE,BURGLARY,DRUG OFFENCES,MISCELLANEOUS CRIMES AGAINST SOCIETY,POSSESSION OF WEAPONS,PUBLIC ORDER OFFENCES,ROBBERY,THEFT,VEHICLE OFFENCES,VIOLENCE AGAINST THE PERSON
0,2010-04-01,230.03125,221.84375,156.00000,18.43750,14.43750,45.75000,87.62500,591.37500,272.15625,364.00000
1,2010-05-01,223.65625,223.25000,160.87500,20.09375,14.43750,50.18750,96.90625,597.50000,278.53125,395.53125
2,2010-06-01,228.43750,223.15625,157.40625,19.65625,13.84375,51.62500,86.81250,610.71875,291.34375,400.93750
3,2010-07-01,227.90625,221.40625,179.31250,21.31250,15.21875,53.43750,83.00000,646.00000,263.96875,404.18750
4,2010-08-01,206.06250,225.06250,192.03125,19.25000,13.81250,45.71875,74.06250,579.65625,255.90625,351.59375
...,...,...,...,...,...,...,...,...,...,...,...
147,2022-07-01,145.03125,115.34375,116.81250,33.25000,16.18750,163.59375,73.21875,616.71875,271.65625,648.34375
148,2022-08-01,142.43750,128.59375,115.62500,32.84375,17.56250,157.28125,69.93750,587.28125,280.09375,622.53125
149,2022-09-01,128.68750,137.28125,96.81250,28.31250,16.06250,137.81250,65.00000,548.31250,285.75000,591.84375
150,2022-10-01,141.25000,143.18750,108.50000,31.09375,16.84375,146.71875,82.15625,659.68750,303.65625,617.46875


In [37]:
merged = df_borough.merge(london_trend, on='Month', suffixes=('', '_LondonAvg'))

# Compute deviation from London average for each category
crime_categories = [col for col in df_borough.columns if col not in ['Borough', 'Month']]
for cat in crime_categories:
    merged[f'{cat}_Deviation'] = merged[cat] - merged[f'{cat}_LondonAvg']

In [39]:
merged = merged.sort_values(['Borough', 'Month'])

for cat in crime_categories:
    merged[f'{cat}_Deviation_Change'] = merged.groupby('Borough')[f'{cat}_Deviation'].diff()

In [43]:
borough_exceptionality = (
    merged.groupby('Borough')[[f'{cat}_Deviation_Change' for cat in crime_categories]]
    .std()
    .sort_values(by=f'{crime_categories[0]}_Deviation_Change', ascending=False)
)

borough_exceptionality

,ARSON AND CRIMINAL DAMAGE_Deviation_Change,BURGLARY_Deviation_Change,DRUG OFFENCES_Deviation_Change,MISCELLANEOUS CRIMES AGAINST SOCIETY_Deviation_Change,POSSESSION OF WEAPONS_Deviation_Change,PUBLIC ORDER OFFENCES_Deviation_Change,ROBBERY_Deviation_Change,THEFT_Deviation_Change,VEHICLE OFFENCES_Deviation_Change,VIOLENCE AGAINST THE PERSON_Deviation_Change
Borough,,,,,,,,,,
Westminster,32.010033,37.221854,78.187001,9.567030,10.849648,26.969972,39.564717,369.903192,48.543791,65.200278
Ealing,29.829384,32.965882,29.795351,9.585905,5.686718,20.913377,14.031185,47.470610,42.406481,45.457658
Croydon,28.167521,34.192612,33.033598,10.218034,7.584943,19.380870,21.579357,59.434666,39.452192,46.563342
Southwark,26.638307,30.811145,43.908701,7.591404,7.765262,18.940609,25.184414,69.970202,44.774770,40.479052
Lambeth,25.576993,33.126350,34.600748,8.688087,8.567699,21.378516,25.586093,84.952050,37.347668,49.064766
Haringey,24.274083,30.051853,33.709652,7.513659,6.814847,14.552249,26.149974,65.636932,48.755276,39.139392
Hillingdon,23.936567,35.112342,28.652667,18.654301,7.689456,18.318626,14.900377,45.667120,44.724094,35.943788
Greenwich,23.311183,26.533434,23.763496,9.497964,6.204031,17.639257,13.216842,47.784883,36.996803,39.987388
Wandsworth,23.027986,30.171244,24.472114,7.477346,5.584543,16.629052,16.206654,54.009231,47.062977,36.603279
